In [25]:
from tweepy import API
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import config

In [33]:
# Client
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter()
        self.twitter_client = API(self.auth)
        self.twitter_user  =twitter_user
    
    def get_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets
    
    def get_frds(self, num_frds):
        frds = []
        for tweet in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_frds):
            frds.append(tweet)
        return frds



# Authenticator
class TwitterAuthenticator():
    def authenticate_twitter(self):
        auth = OAuthHandler(config.API_KEY, config.API_SECRET_KEY)
        auth.set_access_token(config.ACCESS_TOKEN, config.ACCESS_TOKEN_SECRET)
        return auth     


# Streamer
class TwitterStreamer():
    """
    class for streaming and processing live tweets
    """
    def __init__(self):
        self.twitter_auth = TwitterAuthenticator()
        
    def stream_tweets(self, hashtags_list):
        # Handles twitter authentication and connection to twitter streamer API
        listener = TwitterListener()
        auth = self.twitter_auth.authenticate_twitter()
        stream = Stream(auth, listener)
        stream.filter(track=hashtags_list)
        

# Streamer Listener
class TwitterListener(StreamListener):
    """
    Basic listener class that prints to cmd
    """
    def on_data(self, data):
        print(data)
        return True
    
    def on_error(self, status):
        # Return False in case rate limit occurs
        if status==402:
            return False
        print(status)

In [43]:
if __name__ == "__main__":
    hashtags_list = ["avengers"]
#     twitter_streamer = TwitterStreamer()
#     twitter_streamer.stream_tweets(hashtags_list)
    twitter_client = TwitterClient("pycon")
    print(twitter_client.get_frds(10))
    

[User(_api=<tweepy.api.API object at 0x000002582D7CB548>, _json={'id': 863550314, 'id_str': '863550314', 'name': 'Emily Morehouse-Valcarcel', 'screen_name': 'emilyemorehouse', 'location': 'Denver, CO', 'description': '🦑 Director of Engineering @Cuttlesoft. 👩🏻\u200d💻 Python Core Developer. 🐍 @PyCon 202{0,1} chair. 🖤 Lover of programming languages. 🐺 Wife, fur mama. She/her.', 'url': 'https://t.co/oRQK2R5ghm', 'entities': {'url': {'urls': [{'url': 'https://t.co/oRQK2R5ghm', 'expanded_url': 'https://emilyemorehouse.com', 'display_url': 'emilyemorehouse.com', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 3949, 'friends_count': 543, 'listed_count': 96, 'created_at': 'Fri Oct 05 19:23:13 +0000 2012', 'favourites_count': 4333, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 1697, 'lang': None, 'status': {'created_at': 'Wed Aug 19 18:47:55 +0000 2020', 'id': 1296156960966942720, 'id_str': '1296156960